In [806]:
import numpy as np
import pandas as pd
import re

In [807]:
train = pd.read_excel(r'E:\INCEPTEZ\Training\Basics_python\Flight_Price_RandomForest\Flight_Ticket_Participant_Datasets\Data_Train.xlsx')

In [808]:
test = pd.read_excel(r'E:\INCEPTEZ\Training\Basics_python\Flight_Price_RandomForest\Flight_Ticket_Participant_Datasets\Test_set.xlsx')

In [809]:
submission = pd.read_excel(r'E:\INCEPTEZ\Training\Basics_python\Flight_Price_RandomForest\Flight_Ticket_Participant_Datasets\Sample_submission.xlsx')

In [810]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [811]:
test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [812]:
submission.head()

,Price
0,15998
1,16612
2,25572
3,25778
4,16934


In [813]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [814]:
train.describe(include='object').T

,count,unique,top,freq
Airline,10683,12,Jet Airways,3849
Date_of_Journey,10683,44,18/05/2019,504
Source,10683,5,Delhi,4537
Destination,10683,6,Cochin,4537
Route,10682,128,DEL → BOM → COK,2376
Dep_Time,10683,222,18:55,233
Arrival_Time,10683,1343,19:00,423
Duration,10683,368,2h 50m,550
Total_Stops,10682,5,1 stop,5625
Additional_Info,10683,10,No info,8345


In [815]:
train['source']='train'
test['source']='test'

In [816]:
train.shape , test.shape

((10683, 12), (2671, 11))

In [817]:
data = pd.concat([train,test])

In [818]:
data = data.reset_index(drop=True)

In [819]:
data.shape

(13354, 12)

In [820]:
display(data.head(1), data.tail(1))

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,source
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,train


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,source
13353,Multiple carriers,15/06/2019,Delhi,Cochin,DEL → BOM → COK,04:55,19:15,14h 20m,1 stop,No info,NaN,test


### Data Cleaning

In [821]:
data['Airline'].value_counts()

Jet Airways                          4746
IndiGo                               2564
Air India                            2192
Multiple carriers                    1543
SpiceJet                             1026
Vistara                               608
Air Asia                              405
GoAir                                 240
Multiple carriers Premium economy      16
Jet Airways Business                    8
Vistara Premium economy                 5
Trujet                                  1
Name: Airline, dtype: int64

In [822]:
data['Date_of_Journey'].value_counts()

18/05/2019    633
6/06/2019     630
9/05/2019     628
12/06/2019    628
21/05/2019    615
9/06/2019     614
21/03/2019    516
15/05/2019    511
27/05/2019    447
24/06/2019    436
15/06/2019    433
1/06/2019     430
27/06/2019    430
3/06/2019     425
6/03/2019     405
24/03/2019    401
27/03/2019    364
24/05/2019    357
6/05/2019     355
1/05/2019     339
1/04/2019     335
12/05/2019    327
3/03/2019     280
9/03/2019     255
18/03/2019    197
15/03/2019    195
01/03/2019    186
12/03/2019    185
9/04/2019     149
3/04/2019     138
21/06/2019    133
18/06/2019    127
03/03/2019    123
09/03/2019    123
06/03/2019    123
3/05/2019     117
6/04/2019     114
24/04/2019    113
15/04/2019    112
27/04/2019    109
21/04/2019    104
18/04/2019     79
12/04/2019     74
1/03/2019      59
Name: Date_of_Journey, dtype: int64

In [823]:
data['Date_of_Journey'] = pd.to_datetime(data['Date_of_Journey'])

In [824]:
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,source
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,train
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,train
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,train
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,train
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,train


In [825]:
data['Day']= data['Date_of_Journey'].dt.day
data['Year']=data['Date_of_Journey'].dt.year
data['Month']= data['Date_of_Journey'].dt.month

In [826]:
data =data.drop(columns=['Date_of_Journey'])

In [827]:
data['Source'].value_counts()

Delhi       5682
Kolkata     3581
Banglore    2752
Mumbai       883
Chennai      456
Name: Source, dtype: int64

In [828]:
data['Destination'].value_counts()

Cochin       5682
Banglore     3581
Delhi        1582
New Delhi    1170
Hyderabad     883
Kolkata       456
Name: Destination, dtype: int64

In [829]:
data = pd.get_dummies(data , columns=['Airline','Source','Destination'])

In [830]:
data.head()

,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,source,Day,Year,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,train,24,2019,...,0,0,0,0,0,0,0,0,0,1
1,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,train,5,2019,...,0,0,1,0,1,0,0,0,0,0
2,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,train,6,2019,...,0,1,0,0,0,1,0,0,0,0
3,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,train,5,2019,...,0,0,1,0,1,0,0,0,0,0
4,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,train,3,2019,...,0,0,0,0,0,0,0,0,0,1


In [831]:
data['Route'].value_counts()

DEL → BOM → COK                      3000
BLR → DEL                            1941
CCU → BOM → BLR                      1232
CCU → BLR                             896
BOM → HYD                             785
CCU → DEL → BLR                       689
BLR → BOM → DEL                       509
MAA → CCU                             456
DEL → HYD → COK                       411
DEL → JAI → BOM → COK                 314
DEL → BLR → COK                       287
DEL → COK                             262
DEL → AMD → BOM → COK                 246
DEL → MAA → COK                       190
DEL → IDR → BOM → COK                 117
DEL → HYD → MAA → COK                 100
DEL → NAG → BOM → COK                  83
CCU → HYD → BLR                        80
CCU → MAA → BLR                        78
DEL → GOI → BOM → COK                  73
DEL → HYD → BOM → COK                  68
CCU → BBI → BLR                        60
DEL → CCU → BOM → COK                  59
DEL → BHO → BOM → COK             

In [832]:
data['Route'] =data['Route'].apply(str)

In [833]:
data['Route'].apply(lambda x: re.sub('[^A-Za-z]+',',',(x)))

0                        BLR,DEL
1                CCU,IXR,BBI,BLR
2                DEL,LKO,BOM,COK
3                    CCU,NAG,BLR
4                    BLR,NAG,DEL
5                        CCU,BLR
6                    BLR,BOM,DEL
7                    BLR,BOM,DEL
8                    BLR,BOM,DEL
9                    DEL,BOM,COK
10                   DEL,BLR,COK
11                       CCU,BLR
12                       MAA,CCU
13                   CCU,BOM,BLR
14                       CCU,BLR
15               DEL,AMD,BOM,COK
16                   DEL,PNQ,COK
17                   DEL,BOM,COK
18               DEL,CCU,BOM,COK
19                   DEL,BOM,COK
20                   DEL,BOM,COK
21                   BLR,COK,DEL
22                       BLR,DEL
23                       BLR,DEL
24                   CCU,BOM,BLR
25               DEL,IDR,BOM,COK
26                   DEL,LKO,COK
27                   DEL,BOM,COK
28                       BLR,DEL
29                       MAA,CCU
30        

In [834]:
#data['Arrival'] = data['Route'].apply(lambda x:(x.split(',')[-1]))
#data['Departure'] = data['Route'].apply(lambda x:(x.split(',')[0]))

In [835]:
data['Route'].value_counts()

DEL → BOM → COK                      3000
BLR → DEL                            1941
CCU → BOM → BLR                      1232
CCU → BLR                             896
BOM → HYD                             785
CCU → DEL → BLR                       689
BLR → BOM → DEL                       509
MAA → CCU                             456
DEL → HYD → COK                       411
DEL → JAI → BOM → COK                 314
DEL → BLR → COK                       287
DEL → COK                             262
DEL → AMD → BOM → COK                 246
DEL → MAA → COK                       190
DEL → IDR → BOM → COK                 117
DEL → HYD → MAA → COK                 100
DEL → NAG → BOM → COK                  83
CCU → HYD → BLR                        80
CCU → MAA → BLR                        78
DEL → GOI → BOM → COK                  73
DEL → HYD → BOM → COK                  68
CCU → BBI → BLR                        60
DEL → CCU → BOM → COK                  59
DEL → BHO → BOM → COK             

In [836]:
data.head()

,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,source,Day,Year,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,train,24,2019,...,0,0,0,0,0,0,0,0,0,1
1,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,train,5,2019,...,0,0,1,0,1,0,0,0,0,0
2,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,train,6,2019,...,0,1,0,0,0,1,0,0,0,0
3,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,train,5,2019,...,0,0,1,0,1,0,0,0,0,0
4,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,train,3,2019,...,0,0,0,0,0,0,0,0,0,1


In [837]:
data['Duration'] =data['Duration'].apply(lambda x: re.sub('[^0-9]+','.',x))


In [838]:
data['Duration'] =data['Duration'].apply(lambda x: '{0:.1}'.format(x))


In [839]:
data['Duration'] =data['Duration'].astype(int)

In [840]:
data.head()

,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,source,Day,Year,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,BLR → DEL,22:20,01:10 22 Mar,2,non-stop,No info,3897.0,train,24,2019,...,0,0,0,0,0,0,0,0,0,1
1,CCU → IXR → BBI → BLR,05:50,13:15,7,2 stops,No info,7662.0,train,5,2019,...,0,0,1,0,1,0,0,0,0,0
2,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,1,2 stops,No info,13882.0,train,6,2019,...,0,1,0,0,0,1,0,0,0,0
3,CCU → NAG → BLR,18:05,23:30,5,1 stop,No info,6218.0,train,5,2019,...,0,0,1,0,1,0,0,0,0,0
4,BLR → NAG → DEL,16:50,21:35,4,1 stop,No info,13302.0,train,3,2019,...,0,0,0,0,0,0,0,0,0,1


In [841]:
data['Total_Stops'].value_counts()

1 stop      7056
non-stop    4340
2 stops     1899
3 stops       56
4 stops        2
Name: Total_Stops, dtype: int64

In [842]:
data['Total_Stops'] =data['Total_Stops'].replace({'1 stop':1,'non-stop':0,'2 stops':2,'3 stops':3,'4 stops':4})

In [843]:
data['Total_Stops'] =data['Total_Stops'].fillna(0)

In [844]:
data['Total_Stops'] =data['Total_Stops'].astype(int)

In [845]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Route                                      13354 non-null  object 
 1   Dep_Time                                   13354 non-null  object 
 2   Arrival_Time                               13354 non-null  object 
 3   Duration                                   13354 non-null  int32  
 4   Total_Stops                                13354 non-null  int32  
 5   Additional_Info                            13354 non-null  object 
 6   Price                                      10683 non-null  float64
 7   source                                     13354 non-null  object 
 8   Day                                        13354 non-null  int64  
 9   Year                                       13354 non-null  int64  
 10  Month                 

In [846]:
data['Additional_Info'] =[0 if x== 'No info' else 1 for x in data['Additional_Info']]

In [847]:
data['Additional_Info'].value_counts()

0    10493
1     2861
Name: Additional_Info, dtype: int64

In [848]:
data['Route'] = data['Route'].map(data['Route'].value_counts())

In [849]:
data['Arrival_Time'].value_counts()

19:00           536
21:00           466
19:15           434
16:10           185
12:35           159
20:45           141
18:50           138
22:30           136
22:50           134
11:20           112
19:50           112
21:20           109
12:20           100
22:25            97
20:05            90
21:05            86
08:35            85
23:35            84
14:05            83
12:00            81
19:45            76
07:15            73
08:45            73
23:00            71
22:35            71
12:55            70
18:10            68
19:40            66
11:25            65
13:15            65
10:10            65
15:30            65
10:40            64
21:50            63
19:35            63
22:20            63
16:20            62
07:45            62
12:10            62
04:25 07 Jun     61
13:55            61
22:55            61
11:50            60
22:00            60
10:05            59
09:30            59
22:15            58
23:15            58
18:25            57
08:50            57


In [850]:
data['Arrival_Time'] = data['Arrival_Time'].apply(lambda x: re.sub('[^0-9]+','.',x))

In [851]:
data['Arrival_Time'] =data['Arrival_Time'].apply(lambda x: '{0:.5}'.format(x))

In [852]:
data['Arrival_Time'] =data['Arrival_Time'].astype(float)

In [853]:
data['Dep_Time'] =data['Dep_Time'].apply(lambda x: re.sub('[^0-9]+','.',x))

In [854]:
data['Dep_Time'] =data['Dep_Time'].apply(lambda x: '{0:.5}'.format(x))

In [855]:
data['Dep_Time'] =data['Dep_Time'].astype(float)

In [856]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Route                                      13354 non-null  int64  
 1   Dep_Time                                   13354 non-null  float64
 2   Arrival_Time                               13354 non-null  float64
 3   Duration                                   13354 non-null  int32  
 4   Total_Stops                                13354 non-null  int32  
 5   Additional_Info                            13354 non-null  int64  
 6   Price                                      10683 non-null  float64
 7   source                                     13354 non-null  object 
 8   Day                                        13354 non-null  int64  
 9   Year                                       13354 non-null  int64  
 10  Month                 

In [858]:
data.isna().sum()

Route                                           0
Dep_Time                                        0
Arrival_Time                                    0
Duration                                        0
Total_Stops                                     0
Additional_Info                                 0
Price                                        2671
source                                          0
Day                                             0
Year                                            0
Month                                           0
Airline_Air Asia                                0
Airline_Air India                               0
Airline_GoAir                                   0
Airline_IndiGo                                  0
Airline_Jet Airways                             0
Airline_Jet Airways Business                    0
Airline_Multiple carriers                       0
Airline_Multiple carriers Premium economy       0
Airline_SpiceJet                                0


In [859]:
train_preprocessed = data[data['source']=='train']

In [860]:
test_preprocessed = data[data['source']=='test']

In [861]:
train_preprocessed=train_preprocessed.drop(columns=['source'])

In [862]:
test_preprocessed=test_preprocessed.drop(columns=['source','Price'])

In [863]:
train_preprocessed.head()

,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Day,Year,Month,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,1941,22.20,1.10,2,0,0,3897.0,24,2019,3,...,0,0,0,0,0,0,0,0,0,1
1,7,5.50,13.15,7,2,0,7662.0,5,2019,1,...,0,0,1,0,1,0,0,0,0,0
2,46,9.25,4.25,1,2,0,13882.0,6,2019,9,...,0,1,0,0,0,1,0,0,0,0
3,11,18.05,23.30,5,1,0,6218.0,5,2019,12,...,0,0,1,0,1,0,0,0,0,0
4,4,16.50,21.35,4,1,0,13302.0,3,2019,1,...,0,0,0,0,0,0,0,0,0,1


In [864]:
test_preprocessed.head()

,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Day,Year,Month,Airline_Air Asia,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
10683,3000,17.30,4.25,1,1,0,6,2019,6,0,...,0,1,0,0,0,1,0,0,0,0
10684,78,6.20,10.20,4,1,0,5,2019,12,0,...,0,0,1,0,1,0,0,0,0,0
10685,3000,19.15,19.00,2,1,1,21,2019,5,0,...,0,1,0,0,0,1,0,0,0,0
10686,3000,8.00,21.00,1,1,0,21,2019,5,0,...,0,1,0,0,0,1,0,0,0,0
10687,1941,23.55,2.45,2,0,0,24,2019,6,1,...,0,0,0,0,0,0,1,0,0,0


In [865]:
test_preprocessed.reset_index(drop=True, inplace =True)

In [866]:
train_preprocessed.shape , test_preprocessed.shape

((10683, 33), (2671, 32))

In [867]:
train_preprocessed.to_csv('train_preprocessed.csv', index= False)
test_preprocessed.to_csv('test_preprocessed.csv', index= False)